<a href="https://colab.research.google.com/github/Vaibhavs10/scratchpad/blob/main/test_processor_generate_dataset_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install --upgrade transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 23.1 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.0
    Uninstalling transformers-4.40.0:
      Successfully uninstalled transformers-4.40.0


In [2]:
from datasets import load_dataset

In [3]:
ds = load_dataset("EnergyStarAI/ASR")

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [4]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration

In [5]:
processor = WhisperProcessor.from_pretrained("openai/whisper-tiny")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny")

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/151M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.75k [00:00<?, ?B/s]

In [6]:
sample = ds["train"]["audio"][0]

In [7]:
input_features = processor(sample["array"], sampling_rate=sample["sampling_rate"], return_tensors="pt").input_features

In [8]:
predicted_ids = model.generate(input_features)

Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


In [9]:
processor.batch_decode(predicted_ids, skip_special_tokens=True)

[' The lodge in which Oncus was confined was in the very center of the village, and in a situation perhaps more difficult than any other to approach or leave without observation.']

In [10]:
samples = ds["train"]["audio"][0:4]

In [12]:
samples

[{'path': None,
  'array': array([-0.00161743, -0.00158691, -0.00158691, ..., -0.00231934,
         -0.00228882,  0.00164795]),
  'sampling_rate': 16000},
 {'path': None,
  'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         -6.10351562e-05, -3.05175781e-05, -6.10351562e-05]),
  'sampling_rate': 16000},
 {'path': None,
  'array': array([0.28518677, 0.28463745, 0.2097168 , ..., 0.04473877, 0.12634277,
         0.18405151]),
  'sampling_rate': 16000},
 {'path': None,
  'array': array([-0.00033569, -0.00054932, -0.00057983, ...,  0.00061035,
          0.00048828,  0.00033569]),
  'sampling_rate': 16000}]

In [22]:
import torch
model.to("cuda")

def map_to_pred(batch):
    audio = batch["audio"]
    input_features = processor(audio["array"], sampling_rate=audio["sampling_rate"], return_tensors="pt").input_features

    with torch.no_grad():
        predicted_ids = model.generate(input_features.to("cuda"))[0]
    transcription = processor.decode(predicted_ids)
    batch["prediction"] = processor.tokenizer._normalize(transcription)
    return batch

In [15]:
ds = load_dataset("EnergyStarAI/ASR", split="train")

In [23]:
result = ds.map(map_to_pred)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/tokenization_whisper.py:500: UserWarning: The private method `_normalize` is deprecated and will be removed in v5 of Transformers.You can normalize an input string using the Whisper English normalizer using the `normalize` method.
  warnings.warn(


In [25]:
result["prediction"]

['the lodge in which oncus was confined was in the very center of the village and in a situation perhaps more difficult than any other to approach or leave without observation',
 'then came my boy code',
 'the sidewalk right now is i believe there is a 2 foot utility strip and there is a 5 foot sidewalk and then they have stairs that are internal',
 'goobas do not grow on the tops of vines but on the ground on the roots like yams is that so',
 'just as if someone was kissing the window all over outside',
 'yes i think it can all good points on that',
 'recognition in this sense does not necessarily involve more than a habit of association the kind of object we are seeing at the moment is associated with the word cat or with an auditory image of pairing or whatever other characteristic we may happen to recognize in the cat of the moment',
 'celia turned back again to harry',
 'i guess we need the wide wide in their hands',
 'indeed ah',
 'the house was assessed to be worth a £90000 prio